In [2]:
import sys
import os
script_dir = os.getcwd()
sys.path.append(script_dir)

In [3]:
# Importações
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import torch

In [4]:
# Importações
from datasets import load_dataset

# Carregar o dataset com confiança no código remoto
dataset = load_dataset("dair-ai/emotion", trust_remote_code=True)

# Preparar os dados
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
val_texts = dataset['validation']['text']
val_labels = dataset['validation']['label']

# Mapeamento das labels para texto
label_to_text = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

print("Dados de treinamento:", train_texts[:5])
print("Labels de treinamento:", train_labels[:5])
print("Dados de validação:", val_texts[:5])
print("Labels de validação:", val_labels[:5])


Dados de treinamento: ['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property', 'i am feeling grouchy']
Labels de treinamento: [0, 0, 3, 2, 3]
Dados de validação: ['im feeling quite sad and sorry for myself but ill snap out of it soon', 'i feel like i am still looking at a blank canvas blank pieces of paper', 'i feel like a faithful servant', 'i am just feeling cranky and blue', 'i can have for a treat or if i am feeling festive']
Labels de validação: [0, 0, 2, 3, 1]


In [5]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carregar o dataset
dataset = load_dataset("dair-ai/emotion", trust_remote_code=True)

# Definir o comprimento máximo para padding
MAX_LENGTH = 128

# Função de preprocessamento para tokenização
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=MAX_LENGTH)

# Carregar o tokenizer do DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenizar os dados
dataset = dataset.map(tokenize, batched=True)

# Transformar labels para tensores de PyTorch
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Carregar o modelo DistilBERT pré-treinado para classificação
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)



# Mover o modelo para a GPU se disponível
model.to(device)

# Definir a função de cálculo de métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Configuração do treinamento
training_args = TrainingArguments(
    output_dir='./results',          # Diretório para salvar os resultados
    evaluation_strategy="epoch",    # Avaliar o modelo ao final de cada época
    per_device_train_batch_size=8,  # Tamanho do batch de treinamento
    per_device_eval_batch_size=16,  # Tamanho do batch de validação
    num_train_epochs=3,             # Número de épocas de treinamento
    weight_decay=0.01,              # Taxa de decaimento de peso
    logging_dir='./logs',           # Diretório para os logs
    logging_steps=10,
    no_cuda=not torch.cuda.is_available()  # Garantir que a GPU é usada se disponível
)

# Criar o objeto de treinamento do Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics
)

# Treinar o modelo
trainer.train()

# Avaliar o modelo no conjunto de validação
eval_result = trainer.evaluate()

print("Resultados da avaliação:", eval_result)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\vinic\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\vinic\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1540: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/6000 [00:00<?, ?it/s]

{'loss': 1.7445, 'grad_norm': 2.415264368057251, 'learning_rate': 4.991666666666667e-05, 'epoch': 0.01}
{'loss': 1.6507, 'grad_norm': 5.6787872314453125, 'learning_rate': 4.9833333333333336e-05, 'epoch': 0.01}
{'loss': 1.624, 'grad_norm': 4.420863151550293, 'learning_rate': 4.975e-05, 'epoch': 0.01}
{'loss': 1.5432, 'grad_norm': 3.9311673641204834, 'learning_rate': 4.966666666666667e-05, 'epoch': 0.02}
{'loss': 1.5346, 'grad_norm': 10.089386940002441, 'learning_rate': 4.958333333333334e-05, 'epoch': 0.03}
{'loss': 1.3372, 'grad_norm': 7.117930889129639, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.03}
{'loss': 1.3586, 'grad_norm': 7.003167152404785, 'learning_rate': 4.9416666666666664e-05, 'epoch': 0.04}
{'loss': 0.9904, 'grad_norm': 3.350304365158081, 'learning_rate': 4.933333333333334e-05, 'epoch': 0.04}
{'loss': 1.1052, 'grad_norm': 7.905810832977295, 'learning_rate': 4.9250000000000004e-05, 'epoch': 0.04}
{'loss': 0.954, 'grad_norm': 3.312095880508423, 'learning_rate': 4.916

KeyboardInterrupt: 

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset

# Carregar o dataset
dataset = load_dataset("dair-ai/emotion", trust_remote_code=True)

# Preparar os dados
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
val_texts = dataset['validation']['text']
val_labels = dataset['validation']['label']

# Parâmetros
MAX_WORDS = 10000   # Número máximo de palavras a considerar
MAX_LENGTH = 128    # Comprimento máximo das sequências
EMBEDDING_DIM = 100 # Dimensão do embedding

# Tokenização
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

X_train = pad_sequences(train_sequences, maxlen=MAX_LENGTH)
X_val = pad_sequences(val_sequences, maxlen=MAX_LENGTH)

# Transformar labels para one-hot encoding
num_classes = len(set(train_labels))
y_train = to_categorical(train_labels, num_classes=num_classes)
y_val = to_categorical(val_labels, num_classes=num_classes)

# Construir o modelo
model = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_LENGTH),
    LSTM(64, return_sequences=True),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compilar o modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Treinar o modelo
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

# Avaliar o modelo
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=1)
print(f"Validação - Loss: {val_loss}, Acurácia: {val_accuracy}")

# Predição e relatório de classificação
y_pred_probs = model.predict(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_val, axis=1)

print("Relatório de Classificação:")
print(classification_report(y_true, y_pred, target_names=[label_to_text[i] for i in range(num_classes)]))


Epoch 1/10


c:\Users\vinic\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


500/500 ━━━━━━━━━━━━━━━━━━━━ 19s 34ms/step - accuracy: 0.3230 - loss: 1.5901 - val_accuracy: 0.3530 - val_loss: 1.5655
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 17s 34ms/step - accuracy: 0.3571 - loss: 1.4587 - val_accuracy: 0.4600 - val_loss: 1.2595
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 18s 36ms/step - accuracy: 0.5599 - loss: 1.0863 - val_accuracy: 0.7250 - val_loss: 0.8093
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.8042 - loss: 0.6206 - val_accuracy: 0.8180 - val_loss: 0.5655
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 18s 35ms/step - accuracy: 0.8910 - loss: 0.3842 - val_accuracy: 0.8640 - val_loss: 0.4821
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 18s 36ms/step - accuracy: 0.9263 - loss: 0.2709 - val_accuracy: 0.8620 - val_loss: 0.4719
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 18s 35ms/step - accuracy: 0.9461 - loss: 0.2107 - val_accuracy: 0.8685 - val_loss: 0.4588
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 18s 35ms/step - accuracy: 0.9576 - loss: 0.1648 - val_accurac